In [1]:
from Tools_SS2_General import *

In [2]:
df_general=rc('Signaldata.csv',index_col='Unnamed: 0')

In [ ]:
(df_general.mid_kraken-df_general.BBL_10_kraken).plot()

In [60]:
    signalmatrix =df(df_general.mid_kraken-(df_general.MA_10_kraken+2*df_general.STD_10_kraken)).join(
        df(df_general.mid_kraken-(df_general.MA_10_kraken-2*df_general.STD_10_kraken)),rsuffix='r')

In [61]:
signalmatrix

,0,0r
0,-0.779831,2.588165
1,-0.783155,2.586171
2,-0.786461,2.584161
3,-0.789750,2.582134
4,-0.793023,2.580089
...,...,...
300295,-1.500623,2.178573
300296,-1.501730,2.177563
300297,-1.502834,2.176550
300298,-1.513907,2.165540


In [ ]:
df(df_general.mid_kraken-(df_general.MA_10_kraken+2*df_general.STD_10_kraken))

In [21]:
def buy_and_sell_entries_SS2(Entryhandle,Exithandle,df_general=None):
    # df_general is Best Bid Best Ask for both exchanges
    signalmatrix =df(df_general.mid_kraken-(df_general.MA_60_kraken+2*df_general.STD_60_kraken)).join(
        df(df_general.mid_kraken-(df_general.MA_10_kraken)),rsuffix='r')
    signalmatrix.columns=['En','Ex']
    # Our entry and exits
    buyentry=[];sellentry=[]

    # Enter the first buy (Index is zero indexed)
    open_signal = signalmatrix['En']> Entryhandle
    first_entry_idx = df_general[open_signal].head(1).index.values
    buyentry.append(first_entry_idx)

    # Enter the first close
    close_signal = signalmatrix['Ex']<Exithandle
    first_close_signal = df_general[close_signal] 
    first_close_idx = first_close_signal[first_close_signal.index.values > first_entry_idx].head(1).index.values
    sellentry.append(first_close_idx)

    more_oppurtunities = lambda: len(sellentry[-1])!=0
    trade_initated = lambda: len(buyentry[-1])!=0
    
    while more_oppurtunities():
        next_open_signal = df_general[open_signal]
        next_open_idx = next_open_signal[next_open_signal.index.values > sellentry[-1]].head(1).index.values
        buyentry.append(next_open_idx)
        
        if trade_initated():
            next_close_signal = df_general[close_signal] 
            next_close_idx = next_close_signal[next_close_signal.index.values > next_open_idx].head(1).index.values
            sellentry.append(next_close_idx)
        else:
            break

    # Cleanup the mess 
    for i in [sellentry, buyentry]:
        if len(i[-1])==0:
            i.pop(-1)

    return [buyentry, sellentry]

In [51]:
bs=buy_and_sell_entries_SS2(15,0,df_general=df_general)

In [10]:
def trading_plugin_SS2(buysellentries,feesb=0.0025,feess=0.0025,df_price=None):
    
	[buyentry,sellentry]=buysellentries
	Entries=df(buyentry).join(df(sellentry),rsuffix='r')
	Entries.columns=['B','S']

	if len(buyentry)>len(sellentry):
		Entries=Entries.iloc[:(Entries.shape[0]-1),]

	RoT=[None]*(Entries.shape[0])

	for i in range(Entries.shape[0]):
		buyprice=df_price.loc[Entries['B'][i]][2]*(1+feesb)
		sellprice=df_price.loc[Entries['S'][i]][1]*(1-feess)
		RoT[i]=sellprice/buyprice

	Entries_in_time=df(df_price.loc[Entries['B']]['timestamp']).reset_index(
    drop=True).join(df(df_price.loc[Entries['S']]['timestamp']).reset_index(drop=True),rsuffix='S')
	
	Entries_in_time.columns=Entries.columns

	return [Entries_in_time, RoT]


In [11]:
tp=rc('Tradingprices.csv',index_col='Unnamed: 0')

In [52]:
trading_plugin_SS2(bs,feesb=0,feess=0,df_price=tp)

[                     B                    S
 0  2020-07-12 07:58:25  2020-07-12 08:23:39
 1  2020-07-12 13:26:21  2020-07-12 13:30:34
 2  2020-07-12 22:14:04  2020-07-13 06:23:37
 3  2020-07-18 09:03:41  2020-07-18 09:23:19
 4  2020-07-19 16:43:52  2020-07-19 16:54:02,
 [1.0020569697561927,
  0.9865374681992191,
  1.001990043673457,
  0.9998532042525982,
  0.9972837549133178]]